# Potencia necesaria para elevar un helicóptero

In [1]:
import numpy as np
from ipywidgets import IntSlider, FloatSlider, HBox, VBox, Layout
import bqplot.pyplot as plt
import pandas as pd
import math
import bqplot as bq

In [2]:
df = pd.read_csv('Tabla16.csv', decimal=",")

In [3]:
def densidad(h):
    mayores = df.loc[df['Altitud m'] >= h]
    menores = df.loc[df['Altitud m'] <= h]
    mayor = mayores.head(1).iloc[0]
    menor = menores.tail(1).iloc[0]
    if mayor['Altitud m'] == menor['Altitud m']:
        return menor['Densidad kg/m^3']
    
    return menor['Densidad kg/m^3'] + (mayor['Densidad kg/m^3'] - menor['Densidad kg/m^3'])/(mayor['Altitud m'] - menor['Altitud m']) * (h - menor['Altitud m'])

def masa_total(masa_h, masa_p, pasajeros):
    return masa_h + masa_p * pasajeros

def potencia(masa, altura, diametro):
    d = densidad(altura)
    return math.sqrt(((masa * 9.81) ** 3)/(0.7 * d * math.pi * diametro**2))

In [4]:
x = np.linspace(0.0, 20, 10)
potencia_vectorizada = np.vectorize(potencia)
y = potencia_vectorizada(masa_total(800, 20, 0), x, 5)

potencia_fig = plt.figure(title='Potencia en función de la altura')
axes_options = {'x': {'label': 'Altura [mts]'}, 'y': {'label': 'Potencia [W]'}}
potencia_line = plt.plot(x, y, 'm', stroke_width=3, axes_options=axes_options)

In [5]:
layout = Layout(width='30vw', height='4vh') 

masa_h = FloatSlider(description='Masa helicóptero', min=800, max=5000, step=.1, style={'description_width': 'initial'}, layout=layout)
diametro = IntSlider(description='Diámetro aspas', min=5, max=20, style={'description_width': 'initial'}, layout=layout)
pasajeros = IntSlider(description='Número pasajeres', min=1, max=12, style={'description_width': 'initial'}, layout=layout)
masa_p = FloatSlider(description='Masa promedio pasajeres', min=20, max=150, step=.1, style={'description_width': 'initial'}, layout=layout)
altura = FloatSlider(description='Altura máxima', min=1, max=3000, step=.1, style={'description_width': 'initial'}, layout=layout)

slider_layout1 = HBox([masa_h, diametro])
slider_layout2 = HBox([pasajeros, masa_p])
slider_layout3 = HBox([altura])

In [6]:
def update_potencia(change):
    new_masa_h = masa_h.value
    new_diametro = diametro.value
    new_pasajeros = pasajeros.value
    new_masa_p = masa_p.value
    new_altura = altura.value
    new_x = np.linspace(800, masa_total(new_masa_h, new_masa_p, new_pasajeros), 10)
    new_x_altura = np.linspace(0.0, new_altura, 10)
    potencia_line.y = potencia_vectorizada(masa_total(new_masa_h, new_masa_p, new_pasajeros), new_x_altura, 
                                           new_diametro)
    potencia_line.x = new_x_altura
    potencia_line_2.x = new_x
    potencia_line_2.y = potencia_vectorizada(new_x, new_altura, new_diametro)
    
    new_x_diametro = np.linspace(1, new_diametro, 30)
    potencia_line_3.x = new_x_diametro
    potencia_line_3.y = potencia_vectorizada(masa_total(new_masa_h, new_masa_p, new_pasajeros), 
                                             new_altura, new_x_diametro)
    
masa_h.observe(update_potencia, 'value')
diametro.observe(update_potencia, 'value')
pasajeros.observe(update_potencia, 'value')
masa_p.observe(update_potencia, 'value')
altura.observe(update_potencia, 'value')

In [7]:
n = 30
x_peso = np.linspace(800, 2000, 10)
y_peso = potencia_vectorizada(x_peso, n, 5)

layout_2 = Layout(width='33vw', height='60vh')
potencia_fig_2 = plt.figure(title='Potencia en función del peso total', layout=layout_2)
axes_options_2 = {'x': {'label': 'Peso [kg]'}, 'y': {'label': 'Potencia [W]'}}
potencia_line_2 = plt.plot(x_peso, y_peso, 'c', stroke_width=3, axes_options=axes_options_2)

In [8]:
layout_3 = Layout(width='33vw', height='60vh')
x_diametro = np.linspace(1, 10, 30)
y_diametro = potencia_vectorizada(masa_total(800, 20, 0), n, x_diametro)

potencia_fig_3 = plt.figure(title='Potencia en función del diámetro de las aspas', layout=layout_3)
axes_options_3 = {'x': {'label': 'Diámetro [m]'}, 'y': {'label': 'Potencia [W]'}}
potencia_line_3 = plt.plot(x_diametro, y_diametro, 'c', stroke_width=3, axes_options=axes_options_3)

In [9]:
slider_layout4 = HBox([potencia_fig_2, potencia_fig_3])

In [10]:
final_layout = VBox([potencia_fig, slider_layout1, slider_layout2, slider_layout3, slider_layout4])
final_layout